# Tarea 5
- Ivan Morales, 10 de Octubre de 2025

Realizaremos dos experimentos, cada uno con chid experiments

    - Gradient Boost
    - Random Forest

### Importar las librerias necesarias y establecer conexion con Databricks

In [1]:
import os, mlflow
from dotenv import load_dotenv
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer

load_dotenv(override=True) # Cargar las variables de entorno desde el archivo .env
EXPERIMENT_NAME = "/Users/ivan.morales@iteso.mx/nyc-taxi-experiments" 

mlflow.set_tracking_uri("databricks")
experiment = mlflow.set_experiment(experiment_name=EXPERIMENT_NAME)

Preparacion de los datos

In [2]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

df_train_ene = read_dataframe('../data/green_tripdata_2025-01.parquet')
df_train_feb = read_dataframe('../data/green_tripdata_2025-02.parquet')
df_val = read_dataframe('../data/green_tripdata_2025-03.parquet')

In [7]:
df_train_ene.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44218 entries, 0 to 46620
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               44218 non-null  int32         
 1   lpep_pickup_datetime   44218 non-null  datetime64[us]
 2   lpep_dropoff_datetime  44218 non-null  datetime64[us]
 3   store_and_fwd_flag     42295 non-null  object        
 4   RatecodeID             42295 non-null  float64       
 5   PULocationID           44218 non-null  object        
 6   DOLocationID           44218 non-null  object        
 7   passenger_count        42295 non-null  float64       
 8   trip_distance          44218 non-null  float64       
 9   fare_amount            44218 non-null  float64       
 10  extra                  44218 non-null  float64       
 11  mta_tax                44218 non-null  float64       
 12  tip_amount             44218 non-null  float64       
 13  tolls_

In [8]:
df_train = pd.concat([df_train_ene, df_train_feb])
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88436 entries, 0 to 46620
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               88436 non-null  int32         
 1   lpep_pickup_datetime   88436 non-null  datetime64[us]
 2   lpep_dropoff_datetime  88436 non-null  datetime64[us]
 3   store_and_fwd_flag     84590 non-null  object        
 4   RatecodeID             84590 non-null  float64       
 5   PULocationID           88436 non-null  object        
 6   DOLocationID           88436 non-null  object        
 7   passenger_count        84590 non-null  float64       
 8   trip_distance          88436 non-null  float64       
 9   fare_amount            88436 non-null  float64       
 10  extra                  88436 non-null  float64       
 11  mta_tax                88436 non-null  float64       
 12  tip_amount             88436 non-null  float64       
 13  tolls_

Concatemos los datasets de enero y febrero para construir una base mas robusta para validar con los datos de marzo.

Feature Engineering + One Hot Encoding, pipeline simple inicial

In [9]:
def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

X_val = preprocess(df_val, dv)

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

Definir los dataset como objetos de *mlflow* para poderlos trackear

In [10]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2025-01&02")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2025-03")